<a href="https://colab.research.google.com/github/tonykorea99/Prototype/blob/main/KcElectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets
!pip install transformers


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00


In [1]:
import requests
import pandas as pd
import re
from io import StringIO

# 한국어 불용어 리스트 정의
korean_stopwords = ['은', '는', '이', '가', '을', '를', '의', '에', '에서', '와', '과', '도', '로', '으로', '그리고', '하지만',
                    '그러나', '따라서', '그러므로', '그래서', '또한', '매우', '아주', '더', '가장', '너무', '항상', '자주',
                    '때로', '가끔', '이', '그', '저', '나', '너', '우리', '당신', '여러분']

# 불용어 제거 함수 정의
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in korean_stopwords]
    return ' '.join(filtered_words)

# 전처리 함수 정의
def preprocess_text(text):
    # URL 패턴을 [URL]로 대체
    text = re.sub(r'http[s]?://\S+', '[URL]', text)

    # 한국어만 남기고 모두 제거
    text = re.sub(r'[^가-힣\s]', '', text)

    # 불용어 제거
    text = remove_stopwords(text)

    # 문장 길이가 4 이하인 문장 제거
    if len(text.strip()) > 4:
        return text.strip()
    return None

# GitHub에서 CSV 파일을 다운로드하고 전처리하는 함수
def load_and_preprocess_data():
    # GitHub 리포지토리 정보
    user = "tonykorea99"
    repo = "Spam-alart"
    branch = "main"
    directory = "moddata"

    # GitHub API URL
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{directory}?ref={branch}"

    # GitHub API 호출
    response = requests.get(url)
    files = response.json()

    # CSV 파일 목록 필터링
    csv_files = [file for file in files if file['name'].endswith('.csv')]

    # 모든 CSV 파일을 pandas로 불러오고 전처리 적용
    all_data = []
    for file in csv_files:
        raw_url = file['download_url']
        csv_response = requests.get(raw_url)
        df = pd.read_csv(StringIO(csv_response.text))

        # 'v1'과 'v2' 열을 모두 사용하여 전처리 적용
        if 'v1' in df.columns and 'v2' in df.columns:  # 'v1'과 'v2' 열이 있는 경우
            df['processed_text'] = df['v2'].apply(preprocess_text)
        else:
            print(f"{file['name']}에서 'v1' 또는 'v2' 열을 찾을 수 없습니다.")

        # 전처리 후 None 값이 있는 행 제거
        df = df.dropna()

        # 'v1' 열과 'processed_text' 열을 함께 저장
        all_data.append(df[['v1', 'processed_text']])  # 'v1'은 라벨, 'processed_text'는 전처리된 텍스트

    # 모든 데이터를 하나의 데이터프레임으로 병합
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# 데이터 호출 및 전처리 실행
data = load_and_preprocess_data()
print("전처리된 데이터:")
print(data.head())

# 라벨을 추가 (예시: 'spam'을 1, 'ham'을 0으로 변환)
label_mapping = {'spam': 1, 'ham': 0}
data['label'] = data['v1'].map(label_mapping)  # 'v1' 열에서 라벨 매핑


전처리된 데이터:
     v1                                     processed_text
0  spam          국외발신통관세금안내관세미납 원 즉시 납부 바랍니다미납시 강제집행예정민원센터
1  spam              국외발신고객님관세청통관물품 세금확인 관세세금원통관번호자동처리예정문의
2  spam                           발신년 고객님건강검사 통지서 발송완료상세확인
3  spam             국외발신통관세금안내관세미납원 미납시 민사소송 고발조치예정입니다민원접수
4  spam  발신삼도농협택배일정알림 월 추석명절 택배 일정 안내드립니다접수마감 월 일월단식품류는...


In [ ]:
import requests
import pandas as pd
import re
from io import StringIO
from transformers import AutoTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from torch import nn

# 1. 한국어 불용어 리스트 정의
korean_stopwords = ['은', '는', '이', '가', '을', '를', '의', '에', '에서', '와', '과', '도', '로', '으로', '그리고', '하지만',
                    '그러나', '따라서', '그러므로', '그래서', '또한', '매우', '아주', '더', '가장', '너무', '항상', '자주',
                    '때로', '가끔', '이', '그', '저', '나', '너', '우리', '당신', '여러분']

# 불용어 제거 함수 정의
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in korean_stopwords]
    return ' '.join(filtered_words)

# 전처리 함수 정의
def preprocess_text(text):
    # URL 패턴을 [URL]로 대체
    text = re.sub(r'http[s]?://\S+', '[URL]', text)

    # 한국어만 남기고 모두 제거
    text = re.sub(r'[^가-힣\s]', '', text)

    # 불용어 제거
    text = remove_stopwords(text)

    # 문장 길이가 4 이하인 문장 제거
    if len(text.strip()) > 4:
        return text.strip()
    return None

# 2. GitHub에서 CSV 파일을 다운로드하고 전처리하는 함수
def load_and_preprocess_data():
    # GitHub 리포지토리 정보
    user = "tonykorea99"
    repo = "Spam-alart"
    branch = "main"
    directory = "moddata"

    # GitHub API URL
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{directory}?ref={branch}"

    # GitHub API 호출
    response = requests.get(url)
    files = response.json()

    # CSV 파일 목록 필터링
    csv_files = [file for file in files if file['name'].endswith('.csv')]

    # 모든 CSV 파일을 pandas로 불러오고 전처리 적용
    all_data = []
    for file in csv_files:
        raw_url = file['download_url']
        csv_response = requests.get(raw_url)
        df = pd.read_csv(StringIO(csv_response.text))

        # 'v1'과 'v2' 열을 모두 사용하여 전처리 적용
        if 'v1' in df.columns and 'v2' in df.columns:  # 'v1'과 'v2' 열이 있는 경우
            df['processed_text'] = df['v2'].apply(preprocess_text)
            df['label'] = df['v1'].map({'spam': 1, 'ham': 0})  # 라벨 매핑
        else:
            print(f"{file['name']}에서 'v1' 또는 'v2' 열을 찾을 수 없습니다.")

        # 전처리 후 None 값이 있는 행 제거
        df = df.dropna()

        # 'label'과 'processed_text' 열을 함께 저장
        all_data.append(df[['label', 'processed_text']])

    # 모든 데이터를 하나의 데이터프레임으로 병합
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# 3. 데이터 호출 및 전처리 실행
data = load_and_preprocess_data()

# 4. KcElectra 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# num_labels=1로 변경하여 이진 분류 문제로 설정
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=1)

# 데이터셋 준비 (전처리된 데이터를 사용)
def tokenize_function(examples):
    return tokenizer(examples['processed_text'], truncation=True, padding='max_length', max_length=88)

# Dataset 생성 (processed_text와 label을 포함)
train_dataset = Dataset.from_pandas(data[['processed_text', 'label']])

# 토크나이저 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)

# Trainer를 상속받아 compute_loss 메서드를 재정의
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels = labels.unsqueeze(1).float()  # 라벨 크기를 [batch_size, 1]로 변환 및 float 형식으로 변경
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.BCEWithLogitsLoss()  # Sigmoid를 내장한 BCEWithLogitsLoss 사용
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 5. TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    weight_decay=0.01
)

# 6. CustomTrainer를 사용하여 모델 학습
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset
)

# 7. 모델 학습
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/96491 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.062400,0.002133


In [16]:
# 시각화를 위한 로그 저장
train_logs = trainer.state.log_history

# 9. 손실 및 평가지표 시각화
def plot_metrics(logs):
    epochs = [log['epoch'] for log in logs if 'loss' in log]
    losses = [log['loss'] for log in logs if 'loss' in log]

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')
    plt.legend()

    accuracy = [log['eval_accuracy'] for log in logs if 'eval_accuracy' in log]
    precision = [log['eval_precision'] for log in logs if 'eval_precision' in log]
    recall = [log['eval_recall'] for log in logs if 'eval_recall' in log]
    f1 = [log['eval_f1'] for log in logs if 'eval_f1' in log]

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='Accuracy')
    plt.plot(epochs, precision, label='Precision')
    plt.plot(epochs, recall, label='Recall')
    plt.plot(epochs, f1, label='F1-Score')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title('Evaluation Metrics per Epoch')
    plt.legend()

    plt.show()

# 시각화 호출
plot_metrics(train_logs)

Index(['processed_text'], dtype='object')
